1. ### **-- 2346. Compute the Rank as a Percentage**

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, when, abs, round, lit,dense_rank,count
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DateType
from datetime import datetime  # Import datetime module
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("example").getOrCreate()
spark = SparkSession.builder.appName('eer').createOrReplace()
print(spark.sparkContext.getConf().getAll())


stud_schema = StructType([
    StructField('student_id',IntegerType()),
    StructField('department_id',IntegerType()),
    StructField('mark',IntegerType())
])

stud_data = [
[2          , 2             ,650],
[ 8          , 2             , 650],
[ 7          , 1             , 920],
[ 1          , 1             , 610],
[ 3          , 1             , 530]
 ]

stud_df = spark.createDataFrame(schema=stud_schema,data=stud_data)
Window.partitionBy(col('department_id'))
wd = Window.partitionBy(col('department_id')).orderBy(col('mark').desc())
cwd = Window.partitionBy(col('department_id'))
ans = stud_df.withColumn('perc',
                   (dense_rank().over(wd) -1)*100/(count('*').over(cwd)-1)
                  ).withColumn('rnk',dense_rank().over(wd))\
.withColumn('cnt',count("*").over(cwd))

ans.show()

AttributeError: 'Builder' object has no attribute 'createOrReplace'

2. ### **2783. Flight Occupancy and Waitlist Analysis**

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, when, abs, round, lit,dense_rank,count
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DateType
from datetime import datetime  # Import datetime module
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("example").getOrCreate()

Flights_schema = StructType(
   [
    StructField('flight_id',IntegerType()),
    StructField('capacity',IntegerType())
   ]
)

Passengers_schema = StructType([
    StructField('passenger_id',IntegerType()),
    StructField('flight_id',IntegerType()),
])


Flights_data= [
    [1       ,   2        ],
    [ 2     ,     2        ],
    [ 3      ,    1 ]
 ]

Passengers_data = [
[ 101          ,1         ],
[102          ,1         ],
[103          ,1         ],
[104          ,2         ],
[105          ,2         ],
[106          ,3         ],
[107          ,3  ]
]


flights_df = spark.createDataFrame(schema=Flights_schema,data=Flights_data)
Passengers_df = spark.createDataFrame(schema=Passengers_schema,data=Passengers_data)


bookings_df = Passengers_df.groupBy(col('flight_id')).agg(count('*').alias('booked_cnt'))


ans = flights_df.join(bookings_df,'flight_id','left')\
.withColumn('waitlist_cnt',col('booked_cnt')-col('capacity'))


ans.show()




+---------+--------+----------+------------+
|flight_id|capacity|booked_cnt|waitlist_cnt|
+---------+--------+----------+------------+
|        1|       2|         3|           1|
|        3|       1|         2|           1|
|        2|       2|         2|           0|
+---------+--------+----------+------------+



### **3. 550- Game Play Analysis IV**

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, when, abs, round, lit,dense_rank,count,min,datediff
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DateType
from datetime import datetime  # Import datetime module
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("example").getOrCreate()


activity_schema = StructType(
[
    StructField('player_id',IntegerType()),
    StructField('device_id',IntegerType()),
    StructField('event_date',DateType()),
    StructField('games_played',IntegerType())
])

activity_data = [
[ 1,          2,          datetime.strptime('2016-03-01','%Y-%m-%d'),  5            ],
[ 1,          2,          datetime.strptime('2016-03-02','%Y-%m-%d'),  6            ],
[ 2,          3,          datetime.strptime('2017-06-25','%Y-%m-%d'),  1            ],
[ 3,          1,          datetime.strptime('2016-03-02','%Y-%m-%d'),  0            ],
[ 3,          4,          datetime.strptime('2018-07-03','%Y-%m-%d'),  5            ],
[ 3,          3,          datetime.strptime('2016-03-03','%Y-%m-%d'),  2            ]
]

activity_df = spark.createDataFrame(schema=activity_schema,data=activity_data)
activity_df.show()

activity_df.withColumn('diff',
                      datediff(
                        col('event_date'),
                        min('event_date').over(Window.partitionBy('player_id').orderBy('event_date')) 
                      ))\
.where(col('diff')==1).select('player_id').show()

+---------+---------+----------+------------+
|player_id|device_id|event_date|games_played|
+---------+---------+----------+------------+
|        1|        2|2016-03-01|           5|
|        1|        2|2016-03-02|           6|
|        2|        3|2017-06-25|           1|
|        3|        1|2016-03-02|           0|
|        3|        4|2018-07-03|           5|
|        3|        3|2016-03-03|           2|
+---------+---------+----------+------------+

+---------+
|player_id|
+---------+
|        1|
|        3|
+---------+



### **1454 - Active Users**

In [41]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import datediff,lag,ifnull,curdate,count
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DateType
from datetime import datetime  # Import datetime module
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("example").getOrCreate()


logins_schema = StructType(
[
    StructField('id',IntegerType()),
    StructField('login_date',DateType()),
])

data = [
[ 7  ,datetime.strptime('2020-05-30','%Y-%m-%d')],
[ 1  ,datetime.strptime('2020-05-30','%Y-%m-%d')],
    [ 1  ,datetime.strptime('2020-05-30','%Y-%m-%d')],
    [ 1  ,datetime.strptime('2020-05-30','%Y-%m-%d')],
    [ 1  ,datetime.strptime('2020-05-30','%Y-%m-%d')],
    [ 1  ,datetime.strptime('2020-05-30','%Y-%m-%d')],
[ 7  ,datetime.strptime('2020-05-31','%Y-%m-%d')],
[ 7  ,datetime.strptime('2020-06-01','%Y-%m-%d')],
[ 7  ,datetime.strptime('2020-06-02','%Y-%m-%d')],
[ 7  ,datetime.strptime('2020-06-02','%Y-%m-%d')],
[ 7  ,datetime.strptime('2020-06-03','%Y-%m-%d')],
[ 1  ,datetime.strptime('2020-06-07','%Y-%m-%d')],
[ 7  ,datetime.strptime('2020-06-10','%Y-%m-%d')]
]


login_df = spark.createDataFrame(schema=logins_schema,data=data)
login_df.show()

# 5 consecutive days


# method-1

# login_df.distinct('login_date').show()

login_df.dropDuplicates()\
.withColumn('flag',
            ifnull(
            (datediff('login_date',lag('login_date',1,'login_date').over(Window.partitionBy('id').
                                                                   orderBy('login_date'))
               )==1) & 
       (datediff('login_date',lag('login_date',2,'login_date').over(Window.partitionBy('id').
                                                                   orderBy('login_date'))
               )==2) & 
       (datediff('login_date',lag('login_date',3,'login_date').over(Window.partitionBy('id').
                                                                   orderBy('login_date'))
               )==3) & 
       (datediff('login_date',lag('login_date',4,'login_date').over(Window.partitionBy('id').
                                                                   orderBy('login_date'))
               )==4),
                lit(False)
            )
                   )\
.show()
# .where(col('flag')==True)\
# .fillna(col('flag'),'False')\


# method-2

login_df.dropDuplicates().withColumn('rnk',
datediff(curdate(),'login_date')+dense_rank().over(Window.partitionBy('id').orderBy('login_date'))
                   ).groupby('id','rnk').agg(count('*').alias('cnt')).where(col('cnt')>4).show()

+---+----------+
| id|login_date|
+---+----------+
|  7|2020-05-30|
|  1|2020-05-30|
|  1|2020-05-30|
|  1|2020-05-30|
|  1|2020-05-30|
|  1|2020-05-30|
|  7|2020-05-31|
|  7|2020-06-01|
|  7|2020-06-02|
|  7|2020-06-02|
|  7|2020-06-03|
|  1|2020-06-07|
|  7|2020-06-10|
+---+----------+

+---+----------+-----+
| id|login_date| flag|
+---+----------+-----+
|  1|2020-05-30|false|
|  1|2020-06-07|false|
|  7|2020-05-30|false|
|  7|2020-05-31|false|
|  7|2020-06-01|false|
|  7|2020-06-02|false|
|  7|2020-06-03| true|
|  7|2020-06-10|false|
+---+----------+-----+

+---+----+---+
| id| rnk|cnt|
+---+----+---+
|  7|1508|  5|
+---+----+---+



### **2292 - Products With Three or More Orders in Two Consecutive Years**

In [94]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import datediff,lag,ifnull,curdate,count,date_format,col
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DateType
from datetime import datetime  # Import datetime module
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("example").getOrCreate()
from datetime import datetime

orders_schema = StructType(
[
    StructField('order_id',IntegerType()),
    StructField('product_id',IntegerType()),
    StructField('purchase_date',DateType())
]
)

orders_data = [
[1,1,datetime.strptime('2020-03-16','%Y-%m-%d')],
[2,1,datetime.strptime('2020-12-02','%Y-%m-%d')],
[3,1,datetime.strptime('2020-05-10','%Y-%m-%d')],
[4,1,datetime.strptime('2021-12-23','%Y-%m-%d')],
[5,1,datetime.strptime('2021-05-21','%Y-%m-%d')],
[6,1,datetime.strptime('2021-10-11','%Y-%m-%d')],
[7,2,datetime.strptime('2022-10-11','%Y-%m-%d')],
[8,2,datetime.strptime('2023-10-11','%Y-%m-%d')]
]


order_df = spark.createDataFrame(schema=orders_schema,data=orders_data)

cte = order_df.groupby(date_format('purchase_date','y').alias('yr'),'product_id').agg(count('*')\
                                                                                      .alias('cnt'))

cte.show()
cte.printSchema()
# cte.where(col('cnt')>=2).show()

cte1= cte.alias("cte1")
cte2= cte.alias("cte2")

# .where(col('cnt')>=2)
# .where(col('cnt')>=2)

cte1.join(
    (cte2),
          (col("cte1.product_id") == col("cte2.product_id")) &
          (col("cte1.yr")+1 == col("cte2.yr"))&
                (col('cte1.cnt')>=2)&(col('cte2.cnt')>=2)
        ).show()

+----+----------+---+
|  yr|product_id|cnt|
+----+----------+---+
|2021|         1|  3|
|2020|         1|  3|
|2023|         2|  1|
|2022|         2|  1|
+----+----------+---+

root
 |-- yr: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- cnt: long (nullable = false)

+----+----------+---+----+----------+---+
|  yr|product_id|cnt|  yr|product_id|cnt|
+----+----------+---+----+----------+---+
|2020|         1|  3|2021|         1|  3|
+----+----------+---+----+----------+---+



### **2922 - Market Analysis III**

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import datediff,lag,ifnull,curdate,count,date_format,col
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DateType
from datetime import datetime  # Import datetime module
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("example").getOrCreate()
from datetime import datetime

users_sc = StructType([
    StructField('seller_id',IntegerType()),
    StructField('favorite_brand',StringType())    
])

items_sc = StructType([
    StructField('item_id',IntegerType()),
    StructField('item_brand',StringType())
])

orders_sc = StructType(
[
    StructField('orderId',IntegerType()),
    StructField('item_id',IntegerType()),
    StructField('seller_id',IntegerType())
]
)


users_data = [
[1       ,'Lenovo'],
[ 2         ,'Samsung'],
[ 3         ,'LG']
]

items_data = [
    [1       ,'Samsung'],
[ 2       ,'Lenovo'],
[ 3       ,'LG'],
[4       ,'HP']
]


orders_data = [
[1,4       ,2],
[ 2,2       , 3],
[ 3,3       , 3],
[ 4,1       , 2],
[ 5,4       , 2]   
]


users_df = spark.createDataFrame(schema=users_sc,data=users_data)
items_df = spark.createDataFrame(schema=items_sc,data=items_data)
orders_df = spark.createDataFrame(schema=orders_sc,data=orders_data)

orders_df.join(items_df,'item_id').join(users_df,'seller_id').where(col('item_brand')!=col('favorite_brand'))\
.drop_duplicates(['seller_id','item_id'])\
.groupby(col('seller_id')).agg(count('item_id').alias('num_items')).show()

+---------+---------+
|seller_id|num_items|
+---------+---------+
|        3|        1|
|        2|        1|
+---------+---------+



### **1341 - Movie Rating**

In [39]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import max,datediff,lag,ifnull,curdate,count,date_format,col,avg
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DateType
from datetime import datetime  # Import datetime module
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("example").getOrCreate()
from datetime import datetime

users_sc = StructType([
    StructField('user_id',IntegerType()),
    StructField('name',StringType())    
])

users_data =[
[  1           ,'Daniel'],
[ 2            ,'Monica'],
[3           ,'Maria'],
[ 4           ,'James']
]

movieRating_sc = StructType([
    StructField('movie_id',IntegerType()),
    StructField('user_id',IntegerType()),    
    StructField('rating',IntegerType()),
    StructField('created_at',DateType()),
])
movieRating_data = [
[1           , 1            ,3              ,datetime.strptime('2020-01-12','%Y-%m-%d')],
[ 1           , 2            , 4            ,datetime.strptime('2020-02-11','%Y-%m-%d')],
[ 1           , 3            , 2            ,datetime.strptime('2020-02-12','%Y-%m-%d')],
[ 1           , 4            , 1            ,datetime.strptime('2020-01-01','%Y-%m-%d')],
[ 2           , 1            , 5            ,datetime.strptime('2020-02-17','%Y-%m-%d')], 
[ 2           , 2            , 2            ,datetime.strptime('2020-02-01','%Y-%m-%d')], 
[ 2           , 3            , 2            ,datetime.strptime('2020-03-01','%Y-%m-%d')],
[ 3           , 1            , 3            ,datetime.strptime('2020-02-22','%Y-%m-%d')],  
[ 3           , 2            , 4            ,datetime.strptime('2020-02-25','%Y-%m-%d')]
]
    
users_df = spark.createDataFrame(schema=users_sc,data=users_data)    
movieRating_df = spark.createDataFrame(schema=movieRating_sc,data=movieRating_data)    
#     user with max ratings:
cntDF= movieRating_df.groupby(col('user_id')).agg(count('movie_id').alias('cnt')).join(users_df,'user_id').orderBy(col('cnt').desc(),col('name')).limit(1)
cntDF.show()

# how spark handles date_format
movieRating_df.withColumn('YYYY-mm',date_format('created_at','yyyy-M')).limit(1).show()

# movie with highest ratings:
movieRating_df.where(date_format('created_at','yyyy-M')=='2020-2').groupBy('movie_id').agg(avg('rating').alias('avgRatings'))\
.orderBy(col('avgRatings').desc()).limit(1)\
.show()

+-------+---+------+
|user_id|cnt|  name|
+-------+---+------+
|      1|  3|Daniel|
+-------+---+------+

+--------+-------+------+----------+-------+
|movie_id|user_id|rating|created_at|YYYY-mm|
+--------+-------+------+----------+-------+
|       1|      1|     3|2020-01-12| 2020-1|
+--------+-------+------+----------+-------+

+--------+----------+
|movie_id|avgRatings|
+--------+----------+
|       3|       3.5|
+--------+----------+



### **-- 2688 - Find Active Users**

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import max,datediff,lag,ifnull,curdate,count,date_format,col,avg
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DateType
from datetime import datetime  # Import datetime module
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("example").getOrCreate()
from datetime import datetime

users_schema = StructType(
[
    StructField('order_id',IntegerType()),
    StructField('user_id',IntegerType()),
    StructField('item',StringType()),
    StructField('created_at',DateType()),
    StructField('amount',IntegerType())
])


users_data = 


### **2394 - Employees With Deductions**

In [30]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import max,datediff,lag,ifnull,curdate,count,date_format,col,avg,ceil,sum,ifnull,lit,when
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DateType,TimestampType, LongType
from datetime import datetime  # Import datetime module
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("example").getOrCreate()
from datetime import datetime

print(datetime.strptime('2020-02-25 09:00:00','%Y-%m-%d %H:%M:%S'))

employees_schema = StructType(
[
    StructField('employee_id',IntegerType()),
    StructField('needed_hours',IntegerType())
])

logs_schema = StructType(
[
    StructField('employee_id',IntegerType()),
    StructField('in_time',TimestampType())
    ,StructField('out_time',TimestampType())
])

employees_data = [
[1          , 20],
[ 2         , 12],
[ 3         , 2 ]
]

myDateTime = lambda  x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S')

logs_data = [
[1,myDateTime('2022-10-01 09:00:00'),myDateTime('2022-10-01 17:00:00')],
[1,myDateTime('2022-10-06 09:05:04'),myDateTime('2022-10-06 17:09:03')],
[1,myDateTime('2022-10-12 23:00:00'),myDateTime('2022-10-13 03:00:01')],
[2,myDateTime('2022-10-29 12:00:00'),myDateTime('2022-10-29 23:58:58')]
]


# to_timestamp


employees_df = spark.createDataFrame(schema=employees_schema,data=employees_data)
logs_df = spark.createDataFrame(schema=logs_schema,data=logs_data)
logs_df.show()

# logs_df.groupby(col('employee_id')).agg(
#     ceil(
#         sum(
#         col('out_time').cast(LongType()) -
#         col('in_time').cast(LongType()) 
#         )/3600
#     ).alias('diff')
# ).show()

logs_df.groupby(col('employee_id')).agg(
    ceil(
        sum(
        col('out_time').cast(LongType()) -
        col('in_time').cast(LongType()) 
        )/3600
    ).alias('diff')
).join(employees_df,'employee_id','right')\
.where(col('needed_hours')>=when(col('diff').isNull(),0).otherwise(col('diff')))\
.show()

# where(
#     col('needed_hours') >= ( 
#         ifnull(col('diff'),0)
#                            )
#                          ).show()



2020-02-25 09:00:00
+-----------+-------------------+-------------------+
|employee_id|            in_time|           out_time|
+-----------+-------------------+-------------------+
|          1|2022-10-01 09:00:00|2022-10-01 17:00:00|
|          1|2022-10-06 09:05:04|2022-10-06 17:09:03|
|          1|2022-10-12 23:00:00|2022-10-13 03:00:01|
|          2|2022-10-29 12:00:00|2022-10-29 23:58:58|
+-----------+-------------------+-------------------+

+-----------+----+------------+
|employee_id|diff|needed_hours|
+-----------+----+------------+
|          3|NULL|           2|
|          2|  12|          12|
+-----------+----+------------+



### **-- 578. Get Highest Answer Rate Question**

In [33]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import max,datediff,lag,ifnull,curdate,count,date_format,col,avg,ceil,sum,ifnull,lit,when
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DateType,TimestampType, LongType
from datetime import datetime  # Import datetime module
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("example").getOrCreate()
from datetime import datetime

data = [
[ 5  ,'show', 285         , None      , 1      ,123],
[ 5  ,'answer', 285         , 124124    , 1     , 124],
[ 5  ,'show', 369         , None      , 2     , 125],
[ 5  ,'skip', 369         , None      , 2     , 126]
]


survey_df = spark.createDataFrame(data=data,schema=
[
'id'      ,     
 'action'      ,
 'question_id' ,
 'answer_id'    ,
 'q_num'        ,
 'timestamp'    ,
])

survey_df.groupby('question_id')\
.agg(
(sum(when(col('action')=='answer',1).otherwise(0))/sum(when(col('action')=='show',1).otherwise(0))).alias('rate')
)\
.orderBy(col('rate').desc()).limit(1).show()

+-----------+----+
|question_id|rate|
+-----------+----+
|        285| 1.0|
+-----------+----+



### **1205 - Monthly Transactions II**

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import max,datediff,lag,ifnull,curdate,count,date_format,col,avg,ceil,sum,ifnull,lit,when,to_date
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DateType,TimestampType, LongType
from datetime import datetime  # Import datetime module
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("example").getOrCreate()
from datetime import datetime

transactions_data = [
 [101 ,'US'      ,'approved', 1000   ,'2019-05-18'],
 [ 102 ,'US'      ,'declined', 2000   ,'2019-05-19'],
 [ 103 ,'US'      ,'approved', 3000   ,'2019-06-10'],
 [ 104 ,'US'      ,'declined', 4000   ,'2019-06-13'],
 [ 105 ,'US'      ,'approved', 5000   ,'2019-06-15']]

sch = ['id','country','state','amount','trans_date']

transactions_df = spark.createDataFrame(data=transactions_data,schema=sch)
transactions_df = transactions_df.withColumn('trans_date',to_date(col('trans_date'),'yyyy-MM-dd'))
transactions_df.show()
transactions_df.printSchema()
# root
#  |-- id: long (nullable = true)
#  |-- country: string (nullable = true)
#  |-- state: string (nullable = true)
#  |-- amount: long (nullable = true)
#  |-- trans_date: date (nullable = true)

chargebacks_data =  [[102      ,'2019-05-29'],
 [ 101      ,'2019-06-30'],
 [ 105      ,'2019-09-18']]
sch = ['trans_id','trans_date']
chargebacks_df = spark.createDataFrame(data=chargebacks_data,schema=sch)
chargebacks_df = chargebacks_df.withColumn('trans_date',to_date('trans_date','yyyy-MM-dd'))
chargebacks_df.show()
chargebacks_df.printSchema()


chargebacks_info_df = chargebacks_df.join(transactions_df,chargebacks_df['trans_id']==transactions_df['id'])\
.withColumn('state',lit('chargeback'))\
.select('trans_id',col('country'),'state','amount',chargebacks_df['trans_date'])


final_df = transactions_df.unionAll(chargebacks_info_df)
final_df.show()


# for each month and country: 
# the number of approved transactions and their total amount, 
# the number of chargebacks, and their total amount

ans_df = final_df.withColumn('month',date_format('trans_date','yyyy-MM'))\
.groupby('month','country')\
.agg(
    sum(when(col('state')=='approved',1).otherwise(0)).alias('approves'),
    sum(when(col('state')=='approved',col('amount')).otherwise(0)).alias('approved_amount'),
    sum(when(col('state')=='chargeback',1).otherwise(0)).alias('chargebacks'),
    sum(when(col('state')=='chargeback',col('amount')).otherwise(0)).alias('chargeback_amount')
)

ans_df.show()

+---+-------+--------+------+----------+
| id|country|   state|amount|trans_date|
+---+-------+--------+------+----------+
|101|     US|approved|  1000|2019-05-18|
|102|     US|declined|  2000|2019-05-19|
|103|     US|approved|  3000|2019-06-10|
|104|     US|declined|  4000|2019-06-13|
|105|     US|approved|  5000|2019-06-15|
+---+-------+--------+------+----------+

root
 |-- id: long (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)
 |-- amount: long (nullable = true)
 |-- trans_date: date (nullable = true)

+--------+----------+
|trans_id|trans_date|
+--------+----------+
|     102|2019-05-29|
|     101|2019-06-30|
|     105|2019-09-18|
+--------+----------+

root
 |-- trans_id: long (nullable = true)
 |-- trans_date: date (nullable = true)

+---+-------+----------+------+----------+
| id|country|     state|amount|trans_date|
+---+-------+----------+------+----------+
|101|     US|  approved|  1000|2019-05-18|
|102|     US|  declined|  200

### **1070. Product Sales Analysis III**

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import max,datediff,lag,ifnull,curdate,count,date_format,col,avg,ceil,sum,ifnull,lit,when,to_date,dense_rank,min
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DateType,TimestampType, LongType
from datetime import datetime  # Import datetime module
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("example").getOrCreate()
from datetime import datetime

sales_schema = ['sale_id','product_id','year','quantity','price']
data = [
    [ 1       , 100        , 2008 , 10       , 5000],
    [ 2       , 100        , 2009 , 12       , 5000],
    [ 7       , 200        , 2011 , 15       , 9000],
    [ 5       , 100        , 2008 , 10       , 5000]
]
sales_df = spark.createDataFrame(data=data,schema=sales_schema )

# sales_df.withColumn('rnk',dense_rank().over(
#     Window.partitionBy(col('product_id')).orderBy('year')
# )
#                    ).where(col('rnk')==1).dropDuplicates(['product_id']).show()


sales_df.withColumn('rnk',
                   dense_rank().over(Window.partitionBy(col('product_id')).orderBy('year')
                                                       
                                                       )

                   ).where(col('rnk')==1).dropDuplicates(['product_id']).show()





sales_df.groupBy(col('product_id')).agg(
    min(col('year')).alias('firsyYr'),
    max(col('year')).alias('lastYr')    
                                       )\
                                       .show()

+-------+----------+----+--------+-----+---+
|sale_id|product_id|year|quantity|price|rnk|
+-------+----------+----+--------+-----+---+
|      1|       100|2008|      10| 5000|  1|
|      7|       200|2011|      15| 9000|  1|
+-------+----------+----+--------+-----+---+

+----------+-------+------+
|product_id|firsyYr|lastYr|
+----------+-------+------+
|       100|   2008|  2009|
|       200|   2011|  2011|
+----------+-------+------+



### **1098 - Unpopular Books**

In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import max,datediff,lag,ifnull,curdate,count,date_format,col,avg,ceil,sum,ifnull,lit,when,to_date,dense_rank
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DateType,TimestampType, LongType
from datetime import datetime  # Import datetime module
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("example").getOrCreate()
from datetime import datetime

books_schm = ['book_id','name','available_from']
books_data= [
        [1, 'Clark', '2010-01-01'],
[2, 'Dave', '2012-05-12'],
[3, 'Ava', '2019-06-10'],
[4, 'Clark', '2019-06-01'],
[5, 'Clark', '2008-09-21']
]

books_df = spark.createDataFrame(data=books_data,schema=books_schm)
orders_sch = ['order_id','book_id','quantity','dispatch_date']


orders_data=[
[ 1        , 1       , 2        , '2018-07-26'    ],
[ 2        , 1       , 1        , '2018-11-05'    ],
[ 3        , 3       , 8        , '2019-06-11'    ],
[ 4        , 4       , 6        , '2019-06-05'    ],
[ 5        , 4       , 5        , '2019-06-20'    ],
[ 6        , 5       , 9        , '2009-02-02'    ],
[ 7        , 5       , 8        , '2019-04-13'    ]
]

orders_df = spark.createDataFrame(schema=orders_sch,data=orders_data)
orders_df=orders_df.withColumn('dispatch_date',to_date(col('dispatch_date')))
orders_df.printSchema()
orders_df.join(books_df,'book_id','right')\
.withColumn('dispatch_date',when(col('dispatch_date').isNull(),'2019-05-23'
                                ).otherwise(col('dispatch_date')))\
.withColumn('available_from',when(col('available_from').isNull(),'2019-05-23'
                                ).otherwise(col('available_from')))\
.where( (date_format('dispatch_date','yyyy-MM')>='2018-05')
       & 
       (col('available_from')<='2019-05-23')
      )\
.groupby('book_id').agg(sum('quantity').alias('order').alias('cnt')).where(
    (col('cnt').isNull()) |    (col('cnt')<=10)).show()



root
 |-- order_id: long (nullable = true)
 |-- book_id: long (nullable = true)
 |-- quantity: long (nullable = true)
 |-- dispatch_date: date (nullable = true)

+-------+----+
|book_id| cnt|
+-------+----+
|      1|   3|
|      2|NULL|
|      5|   8|
+-------+----+



### **2228 - Users With Two Purchases Within Seven Days**

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import max,datediff,lag,ifnull,curdate,count,date_format,col,avg,ceil,sum,ifnull,lit,when,to_date,dense_rank, concat,collect_list
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DateType,TimestampType, LongType
from datetime import datetime  # Import datetime module
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("example").getOrCreate()
from datetime import datetime

purchases_data = [
    [ 4           , 2       ,'2022-03-13'],
    [ 1           , 5       ,'2022-02-11'],
    [ 3           , 7       ,'2022-06-19'],
    [ 6           , 2       ,'2022-03-20'],
    [ 5           , 7       ,'2022-06-19'],
    [ 2           , 2       ,'2022-06-08']
]
schema = ['purchase_id','user_id','purchase_date']
purchase_df = spark.createDataFrame(schema=schema,data=purchases_data)
purchase_df= purchase_df.withColumn('purchase_date',to_date('purchase_date','yyyy-MM-dd'))
df1 = purchase_df.alias('df1')    
df2 = purchase_df.alias('df2')
# df2 = df2.withColumnRenamed('purchase_date','purchase_dateB')
# .withColumn('diff',datediff(df1['purchase_date'],df2['purchase_dateB']))\
df1.join(df2,col('df1.user_id')==col('df2.user_id'))\
.withColumn('diff',datediff(col('df1.purchase_date'),col('df2.purchase_date')))\
.where(col('diff').between(0,7))\
.groupBy(col('df1.user_id'),df1['purchase_date'])\
.agg(count('df2.purchase_date').alias('countforlast7days'),
    collect_list(col('df2.purchase_date'))
    )\
.where(col('countforlast7days')>1)\
.show(truncate=False)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/19 19:57:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-------+-------------+-----------------+------------------------------------------------+
|user_id|purchase_date|countforlast7days|collect_list(df2.purchase_date)                 |
+-------+-------------+-----------------+------------------------------------------------+
|2      |2022-03-20   |2                |[2022-03-13, 2022-03-20]                        |
|7      |2022-06-19   |4                |[2022-06-19, 2022-06-19, 2022-06-19, 2022-06-19]|
+-------+-------------+-----------------+------------------------------------------------+



### ** 2978 - Symmetric Coordinates**

In [37]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import max,datediff,lag,ifnull,curdate,count,date_format,col,avg,ceil,sum,ifnull,lit,when,to_date,dense_rank, concat,collect_list
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DateType,TimestampType, LongType
from datetime import datetime  # Import datetime module
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("example").getOrCreate()
from datetime import datetime

coords_data = [
[ 20 , 20 ],
[ 20 , 20 ],
[ 20 , 21 ],
[ 23 , 22 ],
[ 22 , 23 ],
[ 21 , 20 ]
]


coords_df = spark.createDataFrame(schema=['x','y'],data=coords_data)
df1 = coords_df.alias('df1')
df2 = coords_df.alias('df2')

df1.join(df2, (col('df1.x')==col('df2.y')) & (col('df1.y')==col('df2.x'))).groupby(df1.x,df1.y).agg(count('*'))\
.where(col('x')<=col('y')).show()
# .dropDuplicates([col('x'),col('y')]).show()



df1.join(df2, (col('df1.x')==col('df2.y')) & (col('df1.y')==col('df2.x'))).select(df1.x,df1.y).drop_duplicates(['x','y'])\
.where(col('x')<=col('y')).show()

# .dropDuplicates([col('x'),col('y')]).show()


+---+---+--------+
|  x|  y|count(1)|
+---+---+--------+
| 20| 20|       4|
| 20| 21|       1|
| 22| 23|       1|
+---+---+--------+

+---+---+
|  x|  y|
+---+---+
| 20| 20|
| 20| 21|
| 22| 23|
+---+---+



### **-- 1107 - New Users Daily Count**

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, expr, min,max,datediff,lag,ifnull,curdate,count,date_format,col,avg,ceil,sum,ifnull,lit,when,to_date,dense_rank, concat,collect_list
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DateType,TimestampType, LongType
from datetime import datetime  # Import datetime module
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("example").getOrCreate()
from datetime import datetime

traffic_data = [
    [1       ,'login','2019-05-01'],
    [1       ,'homepage','2019-05-01'],
    [ 1       ,'logout','2019-05-01'],
    [ 2       ,'login','2019-06-21'],
    [ 2       ,'logout','2019-06-21'],
    [ 3       ,'login','2019-01-01'],
    [ 3       ,'jobs','2019-01-01'],
    [ 3       ,'logout','2019-01-01'],
    [ 4       ,'login','2019-06-21'],
    [ 4       ,'groups','2019-06-21'],
    [ 4       ,'logout','2019-06-21'],
    [ 5       ,'login','2019-03-01'],
    [ 5       ,'logout','2019-03-01'],
    [ 5       ,'login','2019-06-21'],
    [ 5       ,'logout','2019-06-21'],
    [ 5       ,'groups','2019-06-22']
    
]


sch = ['user_id','activity','activity_date']

traffic_df = spark.createDataFrame(schema=sch,data=traffic_data)
traffic_df = traffic_df.withColumn('activity_date',to_date(col('activity_date')))

firstLoginDF = traffic_df.where(col('activity')=='login').groupBy(col('user_id')).agg(min('activity_date').alias('FistLoginDate'))
firstLoginDF.show()
distinctDatesDF = traffic_df.withColumn('cur_date',lit('2019-06-30'))\
.where(
    datediff(col('cur_date'),col('activity_date')).between(0,90))
.dropDuplicates(['activity_date']) #to get the distinct dates


# .distinct()
# .select(expr('distinct activity_date'))


distinctDatesDF.show()


distinctDatesDF.join(firstLoginDF,distinctDatesDF['activity_date']==firstLoginDF['FistLoginDate'],'left')\
.groupBy(distinctDatesDF['activity_date']).agg(count(firstLoginDF['FistLoginDate']).alias('logins')).show()
# count('').show()

# firstLogin_df = traffic_df.where(col('activity')=='login').groupby('user_id')\
# .agg(min(col('activity_date')).alias('firstLoginDate'))

# traffic_df.join(firstLogin_df,'user_id')\
# .withColumn('cur_date',lit('2019-06-30'))\
# .where(
#     (col('activity')=='login') &
#     (col('activity_date')==col('firstLoginDate')) 
#     & datediff(col('cur_date'),col('activity_date')).between(0,90)
# )\
# .groupBy('activity_date').agg(count('*').alias('cnt'))\
# .withColumn('sumSoFar',sum('cnt').over(Window.orderBy('activity_date')))\
# .show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/20 17:10:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-------+-------------+
|user_id|FistLoginDate|
+-------+-------------+
|      1|   2019-05-01|
|      3|   2019-01-01|
|      2|   2019-06-21|
|      5|   2019-03-01|
|      4|   2019-06-21|
+-------+-------------+

+-------+--------+-------------+----------+
|user_id|activity|activity_date|  cur_date|
+-------+--------+-------------+----------+
|      1|   login|   2019-05-01|2019-06-30|
|      1|homepage|   2019-05-01|2019-06-30|
|      1|  logout|   2019-05-01|2019-06-30|
|      2|   login|   2019-06-21|2019-06-30|
|      2|  logout|   2019-06-21|2019-06-30|
|      4|   login|   2019-06-21|2019-06-30|
|      4|  groups|   2019-06-21|2019-06-30|
|      4|  logout|   2019-06-21|2019-06-30|
|      5|   login|   2019-06-21|2019-06-30|
|      5|  logout|   2019-06-21|2019-06-30|
|      5|  groups|   2019-06-22|2019-06-30|
+-------+--------+-------------+----------+

+-------------+------+
|activity_date|logins|
+-------------+------+
|   2019-06-21|    14|
|   2019-05-01|     3|
|   201

### **177 - Nth Highest Salary**

In [38]:
from pyspark.sql import SparkSession
# from pyspark.sql.functions import 
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DateType,TimestampType, LongType
# from datetime import datetime  # Import datetime module
# from pyspark.sql.window import Window
spark = SparkSession.builder.appName("example").getOrCreate()
from datetime import datetime


sch = ['id','salary']
data = [
    [1   ,100    ],
    [ 2   ,200    ],
    [ 3   ,300]
]
n = int(input('Enter number:'))
df = spark.createDataFrame(schema=sch,data=data)
df.dropDuplicates(['salary']).orderBy(col('salary').desc()).offset(n-1).limit(1).show()

Enter number:3
+---+------+
| id|salary|
+---+------+
|  1|   100|
+---+------+



### **1843 - Suspicious Bank Accounts**

In [38]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,sum,date_format,to_timestamp,months_between,lag,when,lit
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("example").getOrCreate()

data =[
    [2              , 3          ,'Creditor', 107100 ,'2021-06-02 11:38:14'],
    [ 4              , 4          ,'Creditor', 10400  ,'2021-06-20 12:39:18'],
    [ 11             , 4          ,'Debtor', 58800  ,'2021-07-23 12:41:55'],
    [ 1              , 4          ,'Creditor', 49300  ,'2021-05-03 16:11:04'],
    [ 15             , 3          ,'Debtor', 75500  ,'2021-05-23 14:40:20'],
    [ 10             , 3          ,'Creditor', 102100 ,'2021-06-15 10:37:16'],
    [ 14             , 4          ,'Creditor', 56300  ,'2021-07-21 12:12:25'],
    [ 19             , 4          ,'Debtor', 101100 ,'2021-05-09 15:21:49'],
    [ 8              , 3          ,'Creditor', 64900  ,'2021-07-26 15:09:56'],
    [ 7              , 3          ,'Creditor',  90900  ,'2021-06-14 11:23:07']
]

trans_df = spark.createDataFrame(schema=['transaction_id','account_id','type','amount','day'],data=data)
trans_df=trans_df.withColumn('day',to_timestamp('day'))

trans_df.select(expr(''))

income_df=trans_df.where(col('type')=='Creditor')\
.withColumn('month',date_format('day','y-MM'))\
.groupby('account_id','month').agg(sum('amount').alias('totAmount'))

data = [[ 3,21000], [ 4, 10400] ]
acc_df = spark.createDataFrame(data=data,schema=['account_id','max_income'])
# acc_df.show()


exceed_df = income_df.join(acc_df,'account_id').where(col('totAmount')>=col('max_income'))\
.withColumn('diff',
           months_between(
               'month',
               lag('month',1,'month').over(Window.partitionBy('account_id').orderBy('month'))
            )
           )

exceed_df.show()
exceed_df.withColumn('diff',
                     when(col('diff').isNull(),0).otherwise(col('diff'))
                    )\
.where(col('diff')>0).show()


+----------+-------+---------+----------+----+
|account_id|  month|totAmount|max_income|diff|
+----------+-------+---------+----------+----+
|         3|2021-06|   300100|     21000|NULL|
|         3|2021-07|    64900|     21000| 1.0|
|         4|2021-05|    49300|     10400|NULL|
|         4|2021-06|    10400|     10400| 1.0|
|         4|2021-07|    56300|     10400| 1.0|
+----------+-------+---------+----------+----+

+----------+-------+---------+----------+----+
|account_id|  month|totAmount|max_income|diff|
+----------+-------+---------+----------+----+
|         3|2021-07|    64900|     21000| 1.0|
|         4|2021-06|    10400|     10400| 1.0|
|         4|2021-07|    56300|     10400| 1.0|
+----------+-------+---------+----------+----+



### **1149 - Article Views II**

In [5]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import count,col
spark = SparkSession.builder.appName("example").getOrCreate()

data = [
    [ 1          , 3         , 5         ,' 2019-08-01' ],
    [ 3          , 4         , 5         ,' 2019-08-01' ],
    [ 1          , 3         , 6         ,' 2019-08-02' ],
    [ 2          , 7         , 7         ,' 2019-08-01' ],
    [ 2          , 7         , 6         ,' 2019-08-02' ],
    [ 4          , 7         , 1         ,' 2019-07-22' ],
    [ 3          , 4         , 4         ,' 2019-07-21' ],
    [ 3          , 4         , 4         ,' 2019-07-21' ]
]
schema = ['article_id','author_id','viewer_id','view_date']
view_df = spark.createDataFrame(data=data,schema=schema)

view_df.distinct().groupby([col('viewer_id'),col('view_date')]).count().where(col('count')>1).show()
# .agg(count(col('article_id')).alias('cnt')).where(col('cnt')>1).show()




view_df.dropDuplicates().groupby('viewer_id','view_date').agg(count('article_id').alias('view')).where(col('view')>1).show()

+---------+-----------+-----+
|viewer_id|  view_date|count|
+---------+-----------+-----+
|        6| 2019-08-02|    2|
|        5| 2019-08-01|    2|
+---------+-----------+-----+

+---------+-----------+----+
|viewer_id|  view_date|view|
+---------+-----------+----+
|        6| 2019-08-02|   2|
|        5| 2019-08-01|   2|
+---------+-----------+----+



# **180. Consecutive Numbers**

In [21]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("example").getOrCreate()

data = [
[1  , 1],
[ 2  , 1],
[ 3  , 1],
[ 4  , 2],
[ 5  , 2],
[ 6  , 2],
[ 7  , 2] ]
from pyspark.sql.functions import lit,when,col,lag,expr
from pyspark.sql.window import Window

df = spark.createDataFrame(schema=['id','num'],data=data)

df1 = df.alias('df1')
df2 = df.alias('df2')
df3 = df.alias('df3')

# df1.join(df2, 
#          (col('df1.id')+1 == col('df2.id')) & 
#          (col('df2.id')+1 == col('df3.id')) & 
#          (col('df1.num') == col('df2.num')) &
#          (col('df2.num') == col('df3.num'))
#         ).distinct().show()

df.withColumn('lag1',
              lag(col('num'),1,'')
              .over(
                  Window.partitionBy(lit('a')).orderBy(col('id'))
                                                )
             ).show()


# df.withColumn('flag',
#               when(
#                   (col('num')==(lag(col('num'),1,-1)
#                     .over(
#                         Window.partitionBy(lit('a')).orderBy(col('id'))
#                          )))&
#                   (col('num')==(lag(col('num'),2,-1)
#                     .over(
#                         Window.partitionBy(lit('a')).orderBy(col('id'))
#                          )))
#                   ,
#                   True
#                    ).otherwise(False)
#              ).where(col('flag')==True).select(expr("num as ConsecutiveNums")).show()

+---+---+----+
| id|num|lag1|
+---+---+----+
|  1|  1|NULL|
|  2|  1|   1|
|  3|  1|   1|
|  4|  2|   1|
|  5|  2|   2|
|  6|  2|   2|
|  7|  2|   2|
+---+---+----+



# Investments in 2016

In [44]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, IntegerType
from pyspark.sql.functions import col,count,sum

spark = SparkSession.builder.appName('inv2016').getOrCreate()
print(spark.sparkContext)

data = [
[ 1   , 10       , 5        , 10  , 10],
[ 2   , 20       , 20       , 20  , 20],
[ 3   , 10       , 30       , 20  , 20],
[ 4   , 10       , 40       , 40  , 40]
]

schema = StructType([
    StructField('pid',IntegerType()),
    StructField('tiv_2015',IntegerType()),
    StructField('tiv_2016',IntegerType()),
    StructField('lat',IntegerType()),
    StructField('lon',IntegerType())
])


df = spark.createDataFrame(data = data, schema=schema)

singleLocDF = df.groupby(col('lat'),col('lon')).agg(count(col('pid')).alias('cnt')).where(col('cnt')==1)


uniqueLocDF  = df.join(singleLocDF,
       (df['lon']==singleLocDF['lon']) &
        (df['lat']==singleLocDF['lat'])
       )

multipletiv2015DF = uniqueLocDF.groupby(col('tiv_2015')).agg(count(col('pid')).alias('cnt')).where(col('cnt')>1)

uniqueLocDF.join(multipletiv2015DF,
                 uniqueLocDF['tiv_2015']==
                 multipletiv2015DF['tiv_2015']

            ).select(sum(col('tiv_2016')).alias('cnt')).show()



# sum('tiv_2016').show()



# df['tiv_2015']
# col('tiv_2015')



<SparkContext master=local[*] appName=inv2016>
+---+
|cnt|
+---+
| 45|
+---+



### Restaurant Growth

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,IntegerType, DateType, StringType
from pyspark.sql.functions import sum


data = [
[1           ,'Jhon'         ,'2019-01-01', 100],
[ 2           ,'Daniel'       ,'2019-01-02', 110],
[ 3           ,'Jade'         ,'2019-01-03', 120],
[ 4           ,'Khaled'       ,'2019-01-04', 130],
[ 5           ,'Winston'      ,'2019-01-05', 110],
[ 6           ,'Elvis'        ,'2019-01-06', 140],
[ 7           ,'Anna'         ,'2019-01-07', 150],
[ 8           ,'Maria'        ,'2019-01-08', 80],
[ 9           ,'Jaze'         ,'2019-01-09', 110],
[ 1           ,'Jhon'         ,'2019-01-10', 130],
[ 3           ,'Jade'         ,'2019-01-10', 150]
]


schema = StructType([
        StructField('customer_id',IntegerType()),
        StructField('name',StringType()),
        StructField('visited_on',DateType()),
        StructField('amount',IntegerType())
])

df = spark.createDataFrame(data = data, schema = schema)

minDateDF = df.select(min(col('visited_on')).alias('minDate'))










### **1532 - The Most Recent Three Orders**

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, StringType, DateType, StructType,StructField,StringType
from datetime import datetime
spark = SparkSession.builder.appName('1532').getOrCreate()

sch = StructType(
[
    StructField('order_id',IntegerType()),
    StructField('order_date',DateType()),
    StructField('customer_id',IntegerType()),
    StructField('cost',IntegerType()),
]
)

convertStrToDT = lambda x:datetime.strptime(x,'%Y-%m-%d')

data = [
[1      ,convertStrToDT('2020-07-31'),  1  ,1   ],
[ 2        ,convertStrToDT('2020-07-30'),  2           , 2 ],         
[ 3        ,convertStrToDT('2020-07-31'),  3           , 3 ],         
[ 4        ,convertStrToDT('2020-07-29'),  4           , 1 ],         
[ 5        ,convertStrToDT('2020-06-10'),  1           , 2 ],         
[ 6        ,convertStrToDT('2020-08-01'),  2           , 1],          
[ 7        ,convertStrToDT('2020-08-01'),  3           , 1],          
[ 8        ,convertStrToDT('2020-08-03'),  1           , 2],          
[ 9        ,convertStrToDT('2020-08-07'),  2           , 3],          
[ 10       ,convertStrToDT('2020-07-15'),  1           , 2]





]
df = spark.createDataFrame(data= data,schema = sch)
df.show()


from pyspark.sql.functions import col,rank

wd = Window.partitionBy('customer_id').orderBy(col('order_date').desc())

df = df.withColumn('rnk',rank().over(wd)).where(col('rnk')<4)


sch = StructType(
[
    StructField('customer_id',IntegerType()),
    StructField('name',StringType())
    
]
)

data = [
[ 1           ,'Winston'],
[ 2           , 'Jonathan'],
[ 3           , 'Annabelle'],
[ 4           , 'Marwan'],
[ 5           , 'Khaled']   
]

df2 = spark.createDataFrame(data=data,schema=sch)

df.join(df2,'customer_id','left').orderBy(col('name'),col('customer_id'),col('order_date').desc()).show()





+--------+----------+-----------+----+
|order_id|order_date|customer_id|cost|
+--------+----------+-----------+----+
|       1|2020-07-31|          1|   1|
|       2|2020-07-30|          2|   2|
|       3|2020-07-31|          3|   3|
|       4|2020-07-29|          4|   1|
|       5|2020-06-10|          1|   2|
|       6|2020-08-01|          2|   1|
|       7|2020-08-01|          3|   1|
|       8|2020-08-03|          1|   2|
|       9|2020-08-07|          2|   3|
|      10|2020-07-15|          1|   2|
+--------+----------+-----------+----+

+-----------+--------+----------+----+---+---------+
|customer_id|order_id|order_date|cost|rnk|     name|
+-----------+--------+----------+----+---+---------+
|          3|       7|2020-08-01|   1|  1|Annabelle|
|          3|       3|2020-07-31|   3|  2|Annabelle|
|          2|       9|2020-08-07|   3|  1| Jonathan|
|          2|       6|2020-08-01|   1|  2| Jonathan|
|          2|       2|2020-07-30|   2|  3| Jonathan|
|          4|       4|2020-07

# 2112 - The Airport With the Most Traffic

In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType,StringType,StructType,StructField

spark = SparkSession.builder.appName('2112').getOrCreate()

sch = StructType(
[
    StructField('departure_airport',IntegerType()),
    StructField('arrival_airport',IntegerType()),
    StructField('flights_count',IntegerType())    
    
])


data = [
    [1                 , 2               , 4  ],
    [2                 , 1               , 5 ],
    [ 2                 , 4               , 5 ]    
]

from pyspark.sql.functions import sum,col,max
df = spark.createDataFrame(data=data,schema=sch)

df = df.select('departure_airport','flights_count').unionAll(df.select('arrival_airport','flights_count')).groupBy('departure_airport').agg(sum(col('flights_count')).alias('totFlights'))

maxFlights = df.select(max('totFlights').alias('maxFlights'))

df.join(maxFlights, df['totFlights']==maxFlights['maxFlights'],'anti').show()



+-----------------+----------+
|departure_airport|totFlights|
+-----------------+----------+
|                1|         9|
|                4|         5|
+-----------------+----------+



# 2142. The Number of Passengers in Each Bus I

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, IntegerType

spark=SparkSession.builder.appName('2142').getOrCreate()

sch = StructType([
    StructField('bus_id',IntegerType()),    
    StructField('arrival_time',IntegerType())
    ]
)
data =[[ 1      , 2            ],
[ 2      , 4            ],
[ 3      , 7]]


busDF = spark.createDataFrame(schema= sch,data=data)

data = [
[11           , 1           ],
[ 12           , 5            ],
[ 13           , 6            ],
[ 14           , 7  ]
]

sch = StructType(
    [
        StructField('passenger_id',IntegerType()),
        StructField('arrival_time',IntegerType())
    ]
)
passDF = spark.createDataFrame(data=data, schema=sch)

from pyspark.sql.functions import count,avg,lag,lit,col


df = busDF.join(passDF,busDF['arrival_time']>=passDF['arrival_time'],'left').groupBy('bus_id').agg(count('passenger_id').alias('psgCnt'))

df = df.join(busDF,'bus_id')

from pyspark.sql.window import Window


wd = Window.orderBy('arrival_time')

df.withColumn('cnt',col('psgCnt') - lag(col('psgCnt'),1,0).over(wd)).drop('psgCnt').show()



24/12/20 14:57:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/20 14:57:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/20 14:57:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/20 14:57:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/20 14:57:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/20 14:57:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/20 1

+------+------------+---+
|bus_id|arrival_time|cnt|
+------+------------+---+
|     1|           2|  1|
|     2|           4|  0|
|     3|           7|  3|
+------+------------+---+



24/12/20 14:57:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/20 14:57:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/20 14:57:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/20 14:57:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


### 1990 - Count the Number of Experiments

In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType

spark = SparkSession.builder.appName('1990').getOrCreate()

sch = StructType([
    StructField('experiment_id',StringType()),
    StructField('platform',StringType()),
    StructField('experiment_name',StringType())
])

data = [
    [ 4             , 'IOS'      , 'Programming'     ],
    [ 13            , 'IOS'      , 'Sports'          ],
    [ 14            , 'Android'  , 'Reading'         ],
    [ 8             , 'Web'      , 'Reading'         ],
    [ 12            , 'Web'      , 'Reading'         ],
    [ 18            , 'Web'      , 'Programming'     ]
]


df = spark.createDataFrame(data=data,schema=sch)

platDF = spark.createDataFrame(schema=['plat'],data = [['IOS'],['Android'],['Web']])
expDF = spark.createDataFrame(data = [['Programming'],['Sports'],['Reading']],schema = ['expName'])


dataDF = platDF.crossJoin(expDF)
from pyspark.sql.functions import col,count
ansDF = dataDF.join(df, (
    (dataDF['plat']==df['platform']) &
    (dataDF['expName']==df['experiment_name'])
                ),
            'left'
           )\
.groupBy(col('plat'),col('expName')).agg(
    count(col('experiment_id'))
                                           )
ansDF.show()
# ,col('dataDF.platform')
                                                      
# .count('experimet_id').show()

+-------+-----------+--------------------+
|   plat|    expName|count(experiment_id)|
+-------+-----------+--------------------+
|    IOS|Programming|                   1|
|    IOS|    Reading|                   0|
|    IOS|     Sports|                   1|
|    Web|Programming|                   1|
|Android|Programming|                   0|
|Android|     Sports|                   0|
|Android|    Reading|                   1|
|    Web|     Sports|                   0|
|    Web|    Reading|                   2|
+-------+-----------+--------------------+



### 1501. Countries You Can Safely Invest In

In [26]:
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.appName('1501').getOrCreate()



dt = lambda x: datetime.strtime(x,'%Y-%m-%d %H:%M:%S')

callSchema = StructType([
    StructField('caller_id',IntegerType()),
    StructField('callee_id',IntegerType()),
    StructField('duration',IntegerType())
])

personSchema = StructType([
    StructField('id',IntegerType()),
    StructField('name',StringType()),
    StructField('phone_number',StringType())
])

countrySchema = StructType([
    StructField('cname',StringType()),
    StructField('country_code',StringType())
])

personData = [
    [ 3  , 'Jonathan' , '051-1234567'],
[ 12 , 'Elvis'    , '051-7654321'],
[ 1  , 'Moncef'   , '212-1234567'],
[ 2  , 'Maroua'   , '212-6523651'],
[ 7  , 'Meir'     , '972-1234567'],
[ 9  , 'Rachel'   , '972-0011100' ]
]


countryData = [
    ['Peru', '051'],
[ 'Israel','972'],
[ 'Morocco','212'],
[ 'Germany','049'],
[ 'Ethiopia','251']
]

callsData = [
[ 1         , 9         , 33       ],
[2         , 9         , 4        ],
[ 1         , 2         , 59       ],
[ 3         , 12        , 102      ],
[ 3         , 12        , 330      ],
[ 12        , 3         , 5        ],
[ 7         , 9         , 13       ],
[ 7         , 1         , 3        ],
[ 9         , 7         , 1        ],
[ 1         , 7         , 7]
]

callsDF = spark.createDataFrame(schema=callSchema,data = callsData)
personsDF = spark.createDataFrame(schema = personSchema,data = personData)
countryDF = spark.createDataFrame(schema=countrySchema,data = countryData)

from pyspark.sql.functions import col, avg,substring,round

allCallsDF = callsDF.select(col('caller_id').alias('id'),'duration')\
.unionAll(callsDF.select('callee_id','duration'))

dataDF = personsDF.join(countryDF,
                       substring(personsDF['phone_number'],1,3)==countryDF['country_code']
                       )

joinedDF = allCallsDF.join(dataDF,'id')

globalAvg = joinedDF.agg(avg('duration').alias('globalAvg')).collect()
print(globalAvg[0]['globalAvg'])
joinedDF.groupby('cname').agg( round(avg('duration'),2).alias('avgCountryWise')).where(col('avgCountryWise')>globalAvg[0]['globalAvg']).show()








55.7
+-----+--------------+
|cname|avgCountryWise|
+-----+--------------+
| Peru|        145.67|
+-----+--------------+



### 2986. Find Third Transaction

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,IntegerType,DoubleType,TimestampType
spark = SparkSession.builder.appName('2986').getOrCreate()
from datetime import datetime

dt = lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S')
data= [
    [ 1       , 65.56  , dt('2023-11-18 13:49:42')],
[ 1       , 96.0   , dt('2023-11-30 02:47:26')],
[ 1       , 7.44   , dt('2023-11-02 12:15:23')],
[ 1       , 49.78  , dt('2023-11-12 00:13:46')],
[ 2       , 40.89  , dt('2023-11-21 04:39:15')],
[ 2       , 100.44 , dt('2023-11-20 07:39:34')],
[ 3       , 37.33  , dt('2023-11-03 06:22:02')],
[ 3       , 13.89  , dt('2023-11-11 16:00:14')],
[ 3       , 7.0    , dt('2023-11-29 22:32:36')]
]


sch = StructType([
    StructField('id',IntegerType()),
    StructField('spend',DoubleType()),
    StructField('transaction_date',TimestampType())
])

df = spark.createDataFrame(schema=sch,data=data)

from pyspark.sql.window import Window
from pyspark.sql.functions import col,lag,rank,when
wd = Window.partitionBy('id').orderBy('transaction_date')

df.withColumn('rnk',rank().over(wd))\
.withColumn('flag', when(
    (col('spend')>lag('spend',1,9999999).over(wd)) & (col('spend')>lag('spend',2,9999999).over(wd)),True
                        ).otherwise(False)
           ).where(
    (col('rnk')==3) & col('flag')
                  ).select(
'id',
    col('spend').alias('third_transaction_spend'),
    col('transaction_date').alias('third_transaction_date')
).show()






+---+-----------------------+----------------------+
| id|third_transaction_spend|third_transaction_date|
+---+-----------------------+----------------------+
|  1|                  65.56|   2023-11-18 13:49:42|
+---+-----------------------+----------------------+



# 1555. Bank Account Summary

In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType, IntegerType

spark = SparkSession.builder.appName('1555').getOrCreate()

userData = [
    [ 1           ,'Moustafa',      100         ],
[ 2           ,'Jonathan',      200       ],
[ 3          ,'Winston',       10000       ],
[ 4           ,'Luis', 800]
]

userSch = StructType(
    [
    StructField('user_id',IntegerType()),
    StructField('user_name',StringType()),
    StructField('credit',IntegerType())
    ]
)

transData = [
    [ 1          , 1          , 3          , 400],    
[ 2          , 3          , 2          , 500],
[ 3          , 2          , 1          , 200 ]
]

transSch = StructType(
    [
    StructField('trans_id',IntegerType()),
    StructField('paid_by',IntegerType()),
    StructField('paid_to',IntegerType()),
    StructField('amount',IntegerType())
    ]
)

userDF = spark.createDataFrame(data=userData,schema = userSch)
userDF.show()
transDF = spark.createDataFrame(data=transData,schema = transSch)
transDF.show()

from pyspark.sql.functions import col,sum,when,ifnull,lit
balDF = transDF.select(col('paid_by').alias('id'),(-col('amount')).alias('amount')).unionAll(
    transDF.select(col('paid_to'),col('amount').alias('amount'))
).groupBy('id').agg(sum('amount').alias('balance'))

balDF.show()

balDF.join(userDF,balDF['id']==userDF['user_id'],'right')\
.withColumn('credit', ifnull(col('balance'),lit(0))+col('credit'))\
.withColumn('breached',when(col('credit')<0,'Yes').otherwise('No'))\
.select('user_id','user_name','credit','breached').show()
 


+-------+---------+------+
|user_id|user_name|credit|
+-------+---------+------+
|      1| Moustafa|   100|
|      2| Jonathan|   200|
|      3|  Winston| 10000|
|      4|     Luis|   800|
+-------+---------+------+

+--------+-------+-------+------+
|trans_id|paid_by|paid_to|amount|
+--------+-------+-------+------+
|       1|      1|      3|   400|
|       2|      3|      2|   500|
|       3|      2|      1|   200|
+--------+-------+-------+------+

+---+-------+
| id|balance|
+---+-------+
|  1|   -200|
|  3|   -100|
|  2|    300|
+---+-------+

+-------+---------+------+--------+
|user_id|user_name|credit|breached|
+-------+---------+------+--------+
|      1| Moustafa|  -100|     Yes|
|      2| Jonathan|   500|      No|
|      3|  Winston|  9900|      No|
|      4|     Luis|   800|      No|
+-------+---------+------+--------+



### 184 - Department Highest Salary

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
spark = SparkSession.builder.appName('184').getOrCreate()

EmployeeSchema = StructType([
    StructField('id',IntegerType()),
    StructField('name',StringType()),
    StructField('salary',IntegerType()),
    StructField('departmentId',IntegerType()),
])
empData = [
    [1  , 'Joe'   , 70000  , 1],
[ 2  , 'Jim'   , 90000  , 1],
[ 3  , 'Henry' , 80000  , 2],
[ 4  , 'Sam'   , 60000  , 2],
[ 5  , 'Max'   , 90000  , 1]
]

DepartmentSchema = StructType(
    [
        StructField('id',IntegerType()),
        StructField('name',StringType())
    ]
)

deptData = [
    [1   ,'IT'],
[ 2  ,'Sales']
]

empDF = spark.createDataFrame(schema=EmployeeSchema,data = empData)
deptDF = spark.createDataFrame(schema=DepartmentSchema,data = deptData)

from pyspark.sql.functions import col,rank
from pyspark.sql.window import Window

wd = Window.partitionBy('departmentId').orderBy(col('salary').desc())

empDF.withColumnRenamed('name','empName').withColumn('rnk',rank().over(wd))\
.where(col('rnk')==1)\
.join(deptDF,empDF['departmentId']==deptDF['id'])\
.select(empDF['id'],'departmentId','empName','name','salary')\
.show()

+---+------------+-------+-----+------+
| id|departmentId|empName| name|salary|
+---+------------+-------+-----+------+
|  5|           1|    Max|   IT| 90000|
|  2|           1|    Jim|   IT| 90000|
|  3|           2|  Henry|Sales| 80000|
+---+------------+-------+-----+------+

